In [1]:
from vivit import run_experiment, test_files, classes
from data import get_video_label, VideoDataGenerator, labels_df, files, labels
from synthetic import *
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import numpy as np
import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
classes = list(labels_df.groupby('label').size().sort_values(ascending=False)[:10].index)
test_files = [f for f in files if labels[int(os.path.basename(f).split('.')[0])] in classes]

In [5]:
base = "../data/videos"
all_files = test_files
for folder in ["rotated", "flipped", "noised", "high_noised"]:
    all_files += [f"{base}/{folder}/{file}" for file in os.listdir(f"{base}/{folder}")]
y = [get_video_label(file) for file in all_files]

train_files, val_files, y_train, y_test = train_test_split(test_files, y, test_size=0.2, stratify=y, random_state=42)
train, validation = VideoDataGenerator(train_files), VideoDataGenerator(val_files)
full = VideoDataGenerator(all_files)

In [6]:
model = run_experiment(data = train, validation = validation, epochs = 100)

2024-04-16 09:46:50.191586: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-04-16 09:46:50.191636: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-16 09:46:50.191644: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-16 09:46:50.191667: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-16 09:46:50.191684: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2024-04-16 09:47:01.251082: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


35/35 ━━━━━━━━━━━━━━━━━━━━ 401s 10s/step - accuracy: 0.0581 - loss: 6.8422 - top-5-accuracy: 0.3012 - val_accuracy: 0.1179 - val_loss: 5.5841 - val_top-5-accuracy: 0.5607
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 308s 9s/step - accuracy: 0.1259 - loss: 5.3143 - top-5-accuracy: 0.5720 - val_accuracy: 0.1071 - val_loss: 4.5939 - val_top-5-accuracy: 0.5679
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 305s 9s/step - accuracy: 0.0965 - loss: 4.3678 - top-5-accuracy: 0.5634 - val_accuracy: 0.1607 - val_loss: 3.7177 - val_top-5-accuracy: 0.5750
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 274s 8s/step - accuracy: 0.1481 - loss: 3.5530 - top-5-accuracy: 0.5547 - val_accuracy: 0.1857 - val_loss: 3.0943 - val_top-5-accuracy: 0.5714
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 317s 9s/step - accuracy: 0.1622 - loss: 2.9929 - top-5-accuracy: 0.5609 - val_accuracy: 0.1607 - val_loss: 2.7445 - val_top-5-accuracy: 0.5714
Epoch 6/100
23/35 ━━━━━━━━━━━━━━━━━━━━ 1:51 9s/step - accuracy: 0.1473 - loss: 2.7172 - top-5-accurac

KeyboardInterrupt: 

In [6]:
model.save("model-small-patch-time.keras")